將google雲端連結到colab目錄中: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

顯示時間，避免追蹤log時誤會:

In [ ]:
def print_date_and_time():
  from datetime import datetime
  import pytz

  now = datetime.now(pytz.timezone("America/New_York"))
  print("=" * 60)
  print(f' Current date and time: {now.strftime("%m/%d/%Y %H:%M:%S")}')
  print("=" * 60)

# example output
print_date_and_time()

確認當前GPU狀態:

In [ ]:
!nvidia-smi

clone espnet:

In [ ]:
# It takes a few seconds
!git clone --depth 5 https://github.com/espnet/espnet

# We use a specific commit just for reproducibility.
%cd /content/espnet
!git checkout 3a22d1584317ae59974aad62feab8719c003ae05

將kaggle指定的訓練資料從google drive搬到espnet/egs2/aishell/asr1底下的download位置:

In [ ]:
%cd /content/espnet/egs2/aishell/asr1
!ls

In [ ]:
%cd /content
!ls

In [ ]:
!ls drive/MyDrive

In [ ]:
!mkdir espnet/egs2/aishell/asr1/downloads

In [ ]:
!ls espnet/egs2/aishell/asr1

In [10]:
!ls espnet/egs2/aishell/asr1/downloads

In [11]:
!cp drive/MyDrive/downloads.zip espnet/egs2/aishell/asr1

In [ ]:
!ls espnet/egs2/aishell/asr1

In [ ]:
!unzip espnet/egs2/aishell/asr1/downloads.zip -d espnet/egs2/aishell/asr1/downloads/

In [ ]:
!ls espnet/egs2/aishell/asr1/downloads

In [15]:
!rm -rf espnet/egs2/aishell/asr1/downloads.zip

In [ ]:
!ls espnet/egs2/aishell/asr1

[註]改aishell/asr1/local/data.sh，將"自動下載訓練資料"和"去除資料空格"功能停掉

[註]改aishell/asr1/conf/train_asr_branchformer.yaml的batch_bins，避免爆炸 調到2.5/25

安裝anaconda和cuda:

In [ ]:
# It takes 30 seconds
%cd /content/espnet/tools
!./setup_anaconda.sh anaconda espnet 3.9

In [ ]:
# It may take 12 minutes
%cd /content/espnet/tools
!make TH_VERSION=1.12.1 CUDA_VERSION=11.6

檢查程式完備性，以下須出現:
[x] torch=1.12.1/
[x] torch cuda=11.6/
[x] torch cudnn=8302/
[x] espnet=202207

In [ ]:
%cd /content/espnet/tools
!. ./activate_python.sh && python3 check_install.py | head -n 40

# NOTE: Checkpoint 1
print_date_and_time()

[註]改home/dl0812509/anaconda3/envs/sepnet/lib/python3.9/site-packages/kaldiio/matio.py line 588: 
char = fd.read(1).decode(encoding=default_encoding,errors = 'ignore')
跳過訓練過程中出現錯誤的檔案

安裝執行aishell時需要的kenlm框架:

In [ ]:
# about 3min
%cd /content/espnet/egs2/aishell/asr1/../../../tools
!make kenlm.done

再次確認訓練資料是否正確擺放:

In [ ]:
%cd /content/espnet/egs2/aishell/asr1
!ls

In [ ]:
!ls downloads

開始訓練，視gpu個數調整code:

In [ ]:
%cd /content/espnet/egs2/aishell/asr1
!./run.sh --ngpu 1